In [5]:
# import json
# import requests

# catalog = json.loads(open("./catalog.json").read())

# for index, item in enumerate(catalog):
#     item['courses'] = requests.get(f"http://adapt2.sis.pitt.edu/next.course-authoring/api/catalog/contents/{item['id']}/courses").json()
#     item['kcs_agg'] = requests.get(f"http://adapt2.sis.pitt.edu/next.course-authoring/api/catalog/contents/aggregate-concepts?contentId={item['id']}").json()
#     item['kcs_um2'] = requests.get(f"http://adapt2.sis.pitt.edu/next.course-authoring/api/catalog/contents/um2-concepts?activityName={item['short_name']}").json()
#     print(f"{index + 1}/{len(catalog)}: {item['short_name']}")
    
# json.dump(catalog, open("./catalog_full.json", "w"), indent=2)

In [6]:
import json

catalog = json.loads(open("./catalog_full.json").read())

all_classifiers = [
    "Added by Jordan on 09-17-25 alice_py_edu_ontology",
    "Arun added 08.12.2025 Arun Parser v0.1",
    "Arun added 08.15.2025 Arun Parser v0.1",
    "Arun parser v0.1",
    "Rafaella added 09.03.2025 alice_edu_py_ontology",
    "acos parser",
    "alice_edu_py_ontology expert",
    "alice_edu_py_ontology jordan",
    "alice_edu_py_ontology llm",
    "alice_edu_py_ontology llm jordan",
    "alice_edu_py_ontology llm mod",
    "alice_edu_py_ontology llm_rully",
    "alice_edu_py_ontology mod",
    "expert",
    "expert_manual",
    "java ontology v2",
    "manual addition",
    "manual_expert",
    "parser",
    "sql concept (code11102002)",
    "sql concept (code2210)",
    "sql concept (code22101)",
    "sql concept (code22120)",
    "sql concept (code2310)",
    "sql concept (code23100)",
    "sql concept (code23101000)",
    "sql concept (code23101001)",
    "sql concept (code231010012)",
    "sql concept (code23121)",
    "sql concept (code3)",
    "sql concept (code31)",
    "sql concept (code3110)",
    "sql concept (code32)",
    "sql concept (code4)",
    "sql concept (code4118)",
    "sql concept (code411841)",
    "sql concept (code411842)",
    "sql concept (code42)",
    "sql concept (code42120)",
    "sql concept (code43)",
    "sql concept (code43101)",
    "sql concept (code43112)",
    "sql concept (code431120)",
    "sql concept (code431130)",
    "sql concept (code431131)",
    "sql concept (code4314)",
    "sql concept (code43150)",
    "sql concept (code4316)",
    "sql concept (code44112)",
    "sql concept (code4611)",
    "sql concept (code4615)",
    "sql_ontology",
    "webex_parser"
]

def group_agg(kcs):
    return {
        **{
            f"{source}": {
                "concepts": [kc['name'] for kc in kcs if kc['source'] == source],
                "note": "",
                "created_at": None,
                "updated_at": None
            }
            for source in all_classifiers
            if len([kc['name'] for kc in kcs if kc['source'] == source]) > 0
        },
        **{
            "unspecified": {
                "concepts": [kc['name'] for kc in kcs if kc['source'] not in all_classifiers],
                "note": "",
                "created_at": None,
                "updated_at": None
            }
        }
    }
    
def group_um2(kcs):
    return {
        **{
            f"{source}": {
                "concepts": [kc['ConceptTitle'] for kc in kcs if kc['ConceptDescription'] == source],
                "note": "",
                "created_at": None,
                "updated_at": None
            }
            for source in all_classifiers
            if len([kc['ConceptTitle'] for kc in kcs if kc['ConceptDescription'] == source]) > 0
        },
        **{
            "unspecified": {
                "concepts": [kc['ConceptTitle'] for kc in kcs if kc['ConceptDescription'] not in all_classifiers],
                "note": "",
                "created_at": None,
                "updated_at": None
            }
        }
    }
    
def merge_dicts(dict1, dict2):
    merged = dict1.copy()
    for key, value in dict2.items():
        if key in merged:
            merged[key]['concepts'] = list(set(merged[key]['concepts'] + value['concepts']))
        else:
            merged[key] = value
    merged = {k: v for k, v in merged.items() if len(v['concepts']) > 0}
    return merged

catalog_v2 = []
for item in catalog:
    new_format = {
        "user_email": "moh70@pitt.edu",
        "status": "public",
        "listed_at": item['creation_date'],
        "updated_at": None,
        "tags": [],
        "identity": {
            "id": item['short_name'],
            "title": item['name'],
            "type": item['type'],
        },
        "links": {
            "demo_url": item['preview_url'],
        },
        "attribution": {
            "created_at": item['creation_date'],
            "provider": item['provider_name'],
            "publisher": "",
            "authors": [
                {
                    "name": item['author_name'],
                    "affiliation": ""
                }
            ]
        },
        "languages": {
            "content_language": "es" if "_es__" in item['short_name'] else "en",
            "programming_languages": [item['domain_name']]
        },
        "content": {
            "prompt": item['problem_statement'],
            "source_code": item['code'],
        },
        "delivery": [
            {"format": "html", 
             "url": item['url']}
        ],
        "rights": {
            "license": "",
            "license_url": "",
            "usage_notes": ""
        },
        "interaction": {
            "interaction_type": "",
        },
        "pedagogy": {
            "learning_objectives": [],
            "instructional_role": "",
            "prerequisites": {
                "topics": [],
                "concepts": [],
                "item_ids": []
            }
        },
        "uses": [
            {
                "context_id": f"cid={course['id']}",
                "context_name": f"{course['domain_name']}: {course['code']} | {course['name']} > {course['unit_name']}",
                "used_at": course['creation_date'],
                "used_by": f"{course['author_name']} - {course['author_affiliation']}"
            }
            for course in item['courses']
        ],
        "classification": {
            "topics": [],
            "difficulty": "",
            "knowledge_components": {
                **merge_dicts(
                    group_agg(item['kcs_agg']),
                    group_um2(item['kcs_um2'])
                )
            }
        },
    }
    catalog_v2.append(new_format)

json.dump(catalog_v2, open("./catalog_reformatted.json", "w"), indent=2)